### Install wikipedia and lxml

In [1]:
!conda install -c conda-forge wikipedia --yes
!conda install -c conda-forge lxml --yes

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



### Get the html source just using pandas read_html

In [2]:
import pandas as pd
import wikipedia as wp
import numpy as np

html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]

### STEP 1 - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [3]:
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### STEP 2 - Ignore cells with a borough that is Not assigned

In [4]:
print("Before: ", df.shape)
df = df[df.Borough != 'Not assigned']
# after drop reset your index
df.reset_index(drop=True,inplace=True)
print("After: ", df.shape)

Before:  (180, 3)
After:  (103, 3)


### STEP 3 - Rows will be combined into one row with the neighborhoods separated with a comma
### There is no need to combine rows as the last version of wikipedia has unique values

In [5]:
df['Neighborhood']=df['Neighborhood'].apply(lambda x: x.replace(' / ', ', '))
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### STEP 4 - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
### There is no such cell in the last version of wikipedia

In [6]:
df.shape

(103, 3)